In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import math

def resetBoundaryConditions(phi):
    phi[0,  :] = 0
    phi[-1, :] = 0
    phi[:,  0] = 0
    phi[:, -1] = 0

def trapezoidIntegration(values, dA):
    return dA * np.sum((values[:-1] + values[1:]) / 2)

# Charge on capacitor plate using Gauss's law
def analyticalCapacitance(phi, C0, gridSize, topPlateY, bottomPlateY, startPlateX, endPlateX, thickness, V):
    # Electric field
    Ex = np.zeros_like(phi)
    Ey = np.zeros_like(phi)
    for j in range(1, gridSize - 1):
        for k in range(1, gridSize - 1):
            if ((topPlateY <= j < topPlateY + thickness or bottomPlateY <= j < bottomPlateY + thickness) and startPlateX <= k < endPlateX): continue # Skip capacitor plates
            Ex[j, k] = -(phi[j, k+1] - phi[j, k-1]) / 2
            Ey[j, k] = -(phi[j+1, k] - phi[j-1, k]) / 2

    dA = 1 # Gaussian-surface differential area (gridpoint units)

    # Electric field along the Gaussian surface
    Ex_TopPlate = Ex[math.floor(topPlateY) - 1 : math.ceil(topPlateY) + thickness + 1, :]
    Ey_TopPlate = Ey[math.floor(topPlateY) - 1 : math.ceil(topPlateY) + thickness + 1, :]

    fluxTop    = trapezoidIntegration(Ey_TopPlate[0, :], dA)
    fluxBottom = trapezoidIntegration(Ey_TopPlate[-1, :], dA)
    fluxSides  = trapezoidIntegration(Ex_TopPlate[:, 0], dA) + trapezoidIntegration(Ex_TopPlate[:, -1], dA)
    totalFlux  = fluxTop + fluxBottom + fluxSides

    capacitorCharge = totalFlux * epsilon_0
    capacitance     = capacitorCharge / V
    return capacitance / C0

def phiPlot(phi):
    x = np.empty(np.shape(phi))
    y = np.empty(np.shape(phi))
    min = phi[0,0]
    max = phi[0,0]
    for i in range(np.shape(phi)[0]):
        for j in range(np.shape(phi)[1]):
            x[i,j] = i
            y[i,j] = j
            if phi[i,j]>max:
                max=phi[i,j]
            if phi[i,j]<min:
                min=phi[i,j]

    level = np.arange(min,max,(max-min)/10)

    plt.figure()
    plt.contour(x,y,phi, levels=level, cmap='plasma')
    plt.colorbar(label='$\Phi(x,y)$')
    plt.title('Contour Plot of $\Phi(x,y)$')
    plt.xlabel('x')
    plt.ylabel('y')

    plt.figure()
    plt.imshow(phi, cmap='plasma')
    plt.colorbar(label='$\Phi(x,y)$')
    plt.title('Density Plot of $\Phi(x,y)$')
    plt.xlabel('x')
    plt.ylabel('y')

    plt.show()
    
def LaplaceEqn2D(D, V, aspectRatio, plot=bool):
    # Discretization of space
    gridSize = D + 1
    phi = np.zeros((gridSize, gridSize))
    plateLength = gridSize // 10
    plateSpace = plateLength * aspectRatio

    # Capacitor plate placement (in gridpoint units)
    thickness    = 1
    topPlateY    = (gridSize // 2) - (plateSpace // 2) - thickness
    bottomPlateY = topPlateY + plateSpace + thickness
    startPlateX  = (gridSize - plateLength) // 2
    endPlateX    = startPlateX + plateLength

    phi[math.floor(topPlateY):math.ceil(topPlateY) + thickness, :]       =  V/2
    phi[math.floor(bottomPlateY):math.ceil(bottomPlateY) + thickness, :] = -V/2

    # Sucessive Over-Relaxation (SOR) method
    for i in range(maxIter):
        maxDiff = 0
        for j in range(1, gridSize - 1):
            for k in range(1, gridSize - 1):
                if ((topPlateY <= j < topPlateY + thickness or bottomPlateY <= j < bottomPlateY + thickness) and startPlateX <= k < endPlateX): continue # Skip capacitor plates
                temp = phi[j, k]
                phi[j, k] = ((1 - omega) * phi[j, k] + omega * 0.25 * (phi[j + 1, k] + phi[j - 1, k] + phi[j, k + 1] + phi[j, k - 1]))
                maxDiff = np.maximum(maxDiff, abs(phi[j, k] - temp))
                resetBoundaryConditions(phi)

        if maxDiff < tolerance:
            break

    if plot == True: phiPlot(phi)

    C0 = epsilon_0 * plateLength / plateSpace
    return analyticalCapacitance(phi, C0, gridSize, topPlateY, bottomPlateY, startPlateX, endPlateX, thickness, V)

In [ ]:
maxIter = 1000
omega = 1.75
tolerance = 1e-8
V = 1
aspectRatio = 0.85 # Problem set specifies 1. 0.85 is used to match Nishiyama and Nakamura's paper for comparison
epsilon_0 = 8.854187817e-12 # Permittivity of free space (F/m)

# Finite-size scaling
D_list = [50, 75, 100, 125, 150, 175, 200, 250, 300]
analyticalCapacitanceList = [LaplaceEqn2D(D, V, aspectRatio, False) for D in D_list]
inverseN = [(1/D) for D in D_list]

slope, intercept, rVal, pVal, stdErr = linregress(inverseN, analyticalCapacitanceList)

plt.figure()
plt.errorbar(inverseN, analyticalCapacitanceList, fmt='o', label='Calculated Capacitances')
plt.plot(inverseN, [slope * x + intercept for x in inverseN], label='Linear Fit')
plt.xlabel('1/N')
plt.ylabel('Analytical Capacitance')
plt.title('Finite-Size Scaling of Capacitance')
plt.legend()
plt.grid(True)
plt.show()
print(f"Extrapolated capacitance at 1/N = 0: {intercept}")

When Nishiyama and Nakamura first simulated a parallel-plate capacitor using a boundary element method, they calculated a normalized capacitance of 1.9759 using an aspect ratio of 0.85. While it is expected for the "real" simulated capacitance to be larger than the idealized case (1.5), their linear extrapolation to determine the y-intercept is large compared to the interval where data-points are easier calculated. This may be seen in Figure 4 of their paper. Here, I have calculated capacitance at these larger grid sizes and have obtained a result of 1.551.  This value is reasonable as it is only slightly larger than the ideal case. The simulation executes in around 16 minutes on my hardware.  To generate contour and density plots, set the Boolean in LaplaceEqn2D to true.